In [1]:
import numpy as np
import pandas as pd
from glob import glob

In [3]:
# data issues, see notes in Bear
bad_data = ['109', '154']

In [5]:
# Most due to failed T1-MNI registrations (more than 10 ROIs with under 5% coverage)
very_bad_cov = ['115', '120', '121', '131', '132', '136', '140', '151', '156', '159', '166', '167', '177']

In [6]:
# cerebellar lesions
cerebellar = ['149','181' ]

In [7]:
skip = bad_data + very_bad_cov + cerebellar

In [8]:
with open('/home/despoB/dlurie/Projects/despolab_lesion/analysis/first_pass_skips.txt', 'w') as outf:
    for pid in skip:
        outf.write(pid)
        outf.write('\n')

In [9]:
superset = set(range(247))

for acq in ['128px','64px']:
    # Load list of subjects with the specified EPI sequence.
    sublist_file = '/home/despoB/dlurie/Projects/despolab_lesion/derivatives/has_acq-{0}.txt'.format(acq)
    subject_list = np.loadtxt(sublist_file, dtype='int')
    subject_list = [str(i) for i in subject_list]
    
    coverage_fpt = '/home/despoB/dlurie/Projects/despolab_lesion/derivatives/sub-{0}/func/sub-{0}_task-rest_acq-{1}_run-01_bold_space-T1w_atlas-BNA_mask-brainmask_EPIcoverage.csv'
    
    for pid in subject_list:
        if pid not in skip:
            cov_data = pd.read_csv(coverage_fpt.format(pid, acq), index_col='roi_label')
            to_keep = set(cov_data[cov_data['pct_cov'] > 25].index.values)
            superset.intersection_update(to_keep)
            #print("Patient {0} has {1} missing ROIs!".format(pid, len(to_keep)))
            

In [10]:
len(superset)

208

In [11]:
all_rois = set(range(247))

In [12]:
to_drop = all_rois - superset

In [13]:
with open('/home/despoB/dlurie/Projects/despolab_lesion/analysis/first_pass_ROIs.txt', 'w') as outf:
    for roi in list(superset):
        outf.write(str(roi))
        outf.write('\n')